In [4]:
import numpy as np
from scipy import stats
import pandas as pd
from scipy import interpolate
from scipy.signal import welch
from biosppy.signals import eda
import os, json


In [5]:
#change work directory to Code
workDir = 'D:/LAB/放鬆-三總/Relaxing/分析'
os.chdir(workDir)

In [6]:
colums_names = ['TimeStamp', 'data']
inputPath = "Data/Sort/"

In [4]:
# def DataSort(filePath):
#     dataList = os.listdir(filePath)
#     allData = pd.DataFrame(columns=colums_names)

#     for dataName in dataList:
#         # N1_VR_1_PreTest_EKG
#         # dataInfo = dataName[:-4].split('_')
        
#         data = pd.read_csv("%s%s"%(filePath, dataName), names=['TimeStamp', 'data'])
#         print(data[0:5])
#         # print(data.info())
#         allData = pd.concat([allData, data], axis=0, ignore_index=True)
#         # print(allData[0:5])

#         # if (dataInfo[4] == "Skin"):
#         #     skinData = pd.read_csv("%s%s"%(filePath, dataName), index_col=0, names=['TimeStamp', 'Data'])

#     allData['orgIndex'] = allData.index
#     allData.sort_values(by=['TimeStamp', 'orgIndex'], inplace=True)
#     # allData.sort_index(inplace=True)
#     allData = allData.drop(columns='orgIndex')
#     print(allData[0:5])

#     return allData

In [5]:
def EkgResDataSort(filePath):
    # EkgData = pd.read_csv("%s%s"%(filePath, 'VR_1_PreTest_EKG.csv'), names=['TimeStamp', 'data'])
    # EkgData['orgIndex'] = EkgData.index
    # ResData = pd.read_csv("%s%s"%(filePath, 'VR_1_PreTest_Res.csv'), names=['TimeStamp', 'data'])
    # ResData['orgIndex'] = ResData.index

    # drop_indexes = []
    # ResInEkg = pd.DataFrame()

    # pre_data = EkgData['data'][0]
    # for index, row in EkgData.iterrows():
    #     if abs(row['data'] - pre_data) > 100:
    #         drop_indexes.append(index)
    #     pre_data = row['data']
        
    # for i in range(0, len(drop_indexes), 2):
    #     for j in range(drop_indexes[i], drop_indexes[i+1]):
    #         ResInEkg = pd.concat([ResInEkg, EkgData.iloc[j]], axis = 0, ignore_index=True)
    #         EkgData.drop(index=j, inplace=True)
    # ResData = pd.concat([ResData, ResInEkg], axis=0, ignore_index=True)

    # ResData.sort_values(by=['TimeStamp', 'orgIndex'], inplace=True)
    # EkgData.sort_values(by=['TimeStamp', 'orgIndex'], inplace=True)
    # # allData.sort_index(inplace=True)
    # ResData = ResData.drop(columns='orgIndex')
    # EkgData = EkgData.drop(columns='orgIndex')
    # print(ResData[0:5])

    # # return allData

    # ResData.to_csv('Data/VR_1_PreTest_Res.csv', index=None, header=None)
    # EkgData.to_csv('Data/VR_1_PreTest_EKG.csv', index=None, header=None)

    
    # EkgData = pd.read_csv("%s%s"%(filePath, 'VR_1_PreTest_EKG.csv'), names=['TimeStamp', 'data'])
    # ResData = pd.read_csv("%s%s"%(filePath, 'VR_1_PreTest_Res.csv'), names=['TimeStamp', 'data'])
    # ResData['original_index'] = ResData.index

    # # 計算數據的差異值
    # EkgData['diff'] = EkgData['data'].diff().fillna(0)

    # # 使用布林索引找到需要刪除的數據
    # drop_mask = EkgData['diff'].abs() > 100
    # drop_data = EkgData[drop_mask].copy()
    # drop_data['original_index'] = drop_data.index

    # # 移除不需要的數據
    # EkgData.drop(columns=['diff'], inplace=True)
    # EkgData = EkgData[~drop_mask]
    # ResData = pd.concat([ResData, drop_data[['TimeStamp', 'data', 'original_index']]], axis=0, ignore_index=True)

    # ResData.sort_values(by=['TimeStamp', 'original_index'], inplace=True)
    # # EkgData.sort_values(by='TimeStamp', inplace=True)
    
    # ResData.drop(columns='original_index', inplace=True, errors='ignore') # Use errors='ignore' to avoid KeyError in case the column doesn't exist.

    # ResData.to_csv('Data/VR_1_PreTest_Res.csv', index=None, header=None)
    # EkgData.to_csv('Data/VR_1_PreTest_EKG.csv', index=None, header=None)


    EkgData = pd.read_csv("%s%s"%(filePath, 'VR_1_PreTest_EKG.csv'), names=['TimeStamp', 'data'])
    ResData = pd.read_csv("%s%s"%(filePath, 'VR_1_PreTest_Res.csv'), names=['TimeStamp', 'data'])

    EkgData['diff'] = EkgData['data'].diff().fillna(0)
    # 標記異常值
    anomaly_mask = EkgData['diff'].abs() > 100

    # 識別連續的異常段
    anomaly_segments = []
    start_index = 0
    for i, is_anomaly in anomaly_mask.items():
        if is_anomaly:
            print(i,start_index)
            if start_index == 0 :  # 開始新的異常段
                start_index = i
            else:
                anomaly_segments.append((start_index, i-1))
                start_index = 0

    if start_index != 0:  # 最後一段異常段
        anomaly_segments.append((start_index, len(EkgData)-1))


    for start, end in anomaly_segments:
        anomaly_data = EkgData.iloc[start:end+1].copy()
        anomaly_data['original_index'] = anomaly_data.index
        ResData = pd.concat([ResData, anomaly_data], axis=0, ignore_index=True)
        EkgData.drop(index=range(start, end+1), inplace=True)

    # 移除不需要的數據
    EkgData.drop(columns=['diff'], inplace=True)

    ResData.sort_values(by=['TimeStamp', 'original_index'], inplace=True)
    #EkgData.sort_values(by='TimeStamp', inplace=True)
    ResData.drop(columns=['diff','original_index'], inplace=True)

    ResData.to_csv('Data/Sort/VR_1_PreTest_Res.csv', index=None, header=None)
    EkgData.to_csv('Data/Sort/VR_1_PreTest_EKG.csv', index=None, header=None)

In [11]:
def detect_anomalies(data_df, type):
    data_df['diff'] = data_df['data'].diff().fillna(0)
    anomaly_mask = data_df['diff'].abs() > 93
    data_df.drop(columns=['diff'], inplace=True)
    
    anomaly_segments = []
    start_index = None
    for i, is_anomaly in enumerate(anomaly_mask):
        if is_anomaly:
            if  start_index is None:
                start_index = i
            else:
                if (type and (i-start_index > 2) and (i-start_index < 7)) or type == 0:
                    anomaly_segments.append((start_index, i-1))
                    start_index = None
                else:
                    start_index = i
    if start_index is not None:
        anomaly_segments.append((start_index, len(data_df)-1))
    
    return anomaly_segments


def process_data(source_df,extract_df,type):
    anomaly_segments = detect_anomalies(source_df, type)
    for start, end in  reversed(anomaly_segments):
        anomaly_data = source_df.iloc[start:end+1].copy()
        extract_df = pd.concat([extract_df, anomaly_data], axis=0, ignore_index=True)
        source_df.drop(index=range(start, end+1), inplace=True)

    return source_df,extract_df

def main(filePath):
    EkgData = pd.read_csv("%s%s"%(filePath, 'A36_VR_1_PostTest_EKG.csv'), names=['TimeStamp', 'data'])
    ResData = pd.read_csv("%s%s"%(filePath, 'A36_VR_1_PostTest_Res.csv'), names=['TimeStamp', 'data'])
    # EkgData = pd.read_csv("%s%s"%(filePath, 'A36_VR_1_PostTest_EKG_new.csv'), names=['TimeStamp', 'data'])
    # ResData = pd.read_csv("%s%s"%(filePath, 'A36_VR_1_PostTest_Res_new.csv'), names=['TimeStamp', 'data'])
    EkgData["orginal_index"] = range(1, len(EkgData) + 1)
    ResData["orginal_index"] = range(1, len(ResData) + 1)
    ResExtractDf = pd.DataFrame()

    EkgExtractDf = pd.DataFrame()

    ResData,ResExtractDf = process_data(ResData,ResExtractDf,0)
    EkgData,EkgExtractDf = process_data(EkgData,EkgExtractDf,1)

    EkgData = pd.concat([EkgData, ResExtractDf], axis=0, ignore_index=True)
    ResData = pd.concat([ResData, EkgExtractDf], axis=0, ignore_index=True)

    EkgData.sort_values(by=['TimeStamp', 'orginal_index'], inplace=True)
    ResData.sort_values(by=['TimeStamp', 'orginal_index'], inplace=True)

    # ResData,ResExtractDf= process_data(ResData,ResExtractDf)
    # EkgData = pd.concat([EkgData, ResExtractDf], axis=0, ignore_index=True)
    # EkgData.sort_values(by=['TimeStamp', 'orginal_index'], inplace=True)
    # EkgData,EkgExtractDf = process_data(EkgData,EkgExtractDf)
    # ResData = pd.concat([ResData, EkgExtractDf], axis=0, ignore_index=True)
    # ResData.sort_values(by=['TimeStamp', 'orginal_index'], inplace=True)

    EkgData.drop(columns=['orginal_index'], inplace=True)
    ResData.drop(columns=['orginal_index'], inplace=True)


    ResData.to_csv('Data/Sort/A36_VR_1_PostTest_Res_new.csv', index=None, header=None)
    EkgData.to_csv('Data/Sort/A36_VR_1_PostTest_EKG_new.csv', index=None, header=None)

def checkIsValid(filePath):
    print(filePath)
    data_df = pd.read_csv(filePath, names=['TimeStamp', 'data'])
    data_df['diff'] = data_df['data'].diff().fillna(0)
    #print(data_df[69050:69080])
    anomaly_mask = data_df['diff'].abs() > 93
    
    for i, is_anomaly in enumerate(anomaly_mask):
        if is_anomaly:
            print(i+1)



main(inputPath)

checkIsValid('Data/Sort/A36_VR_1_PostTest_Res_new.csv')
checkIsValid('Data/Sort/A36_VR_1_PostTest_EKG_new.csv')

Data/Sort/A36_VR_1_PostTest_Res_new.csv
Data/Sort/A36_VR_1_PostTest_EKG_new.csv


In [12]:
def checkRIsValid(filePath):
    print(filePath)
    data_df = pd.read_csv(filePath, names=['TimeStamp', 'data'])
    data_df['diff'] = data_df['data'].diff().fillna(0)
    # anomaly_mask = data_df['diff'].abs() > 93
    anomaly_mask = data_df['diff'].abs() > 50   #A01 A02
    
    for i, is_anomaly in enumerate(anomaly_mask):
        # if (is_anomaly and i > 106000):
        if (is_anomaly):
            print(i+1)

checkRIsValid('Data/Sort/A36_VR_1_PostTest_Res_new.csv')
checkRIsValid('Data/Sort/A36_VR_1_PostTest_EKG_new.csv')

Data/Sort/A36_VR_1_PostTest_Res_new.csv
Data/Sort/A36_VR_1_PostTest_EKG_new.csv


In [2]:
def DataSort(filePath, prefix):
    dataList = os.listdir(filePath)
    dataName = dataList[0][:-7]
    rsData = pd.DataFrame(columns=colums_names)
    esData = pd.DataFrame(columns=colums_names)

    ekgRawData = pd.read_csv("%s%s%s"%(filePath, dataName, 'EKG.csv'), names=['TimeStamp', 'Data'])
    resRawData = pd.read_csv("%s%s%s"%(filePath, dataName, 'Res.csv'), names=['TimeStamp', 'Data'])
    skinRawData = pd.read_csv("%s%s%s"%(filePath, dataName, 'Skin.csv'), names=['TimeStamp', 'Data'])

    rsData = pd.concat([resRawData, skinRawData], axis=0, ignore_index=True)
    rsData['orgIndex'] = rsData.index
    rsData.sort_values(by=['TimeStamp', 'orgIndex'], inplace=True)
    rsData = rsData.drop(columns='orgIndex')
    resData = rsData[rsData.Data > -2000]
    print(resData[0:5])
    skinData = rsData[rsData.Data < -2000]
    print(skinData[0:5])

    esData = pd.concat([ekgRawData, skinData], axis=0, ignore_index=True)
    esData['orgIndex'] = esData.index
    esData.sort_values(by=['TimeStamp', 'orgIndex'], inplace=True)
    esData = esData.drop(columns='orgIndex')
    print(esData[0:5])
    ekgData = esData[esData.Data > -2000]
    print(ekgData[0:5])
    skinData = esData[esData.Data < -2000]
    print(skinData[0:5])

    skinData.to_csv('Data/Sort/' + prefix + dataName + 'Skin.csv', index=None, header=None)
    resData.to_csv('Data/Sort/' + prefix + dataName + 'Res.csv', index=None, header=None)
    ekgData.to_csv('Data/Sort/' + prefix + dataName + 'EKG.csv', index=None, header=None)

In [10]:
# allSortData = DataSort(inputPath)
# EkgResDataSort(inputPath)
# main(inputPath)
DataSort(inputPath, 'A36_')

# EKG資料有到300多?
# RES看起來400以上?
# Skin < -1000
# -1000 < RES < -100 & 100 < RES < 1000
# -100 < EKG < 100

# skinData = allSortData[allSortData.data < -1000]
# print(skinData[0:5])
# resData = allSortData[((allSortData.data < -100) & (allSortData.data > -1000)) | ((allSortData.data > 100) & (allSortData.data < 1000))]
# resData = allSortData[allSortData.data > -1000]
# print(resData[0:5])
# ekgData = allSortData[(allSortData.data > -100) & (allSortData.data < 100)]
# ekgData = allSortData[allSortData.data > -1000]
# print(ekgData[0:5])

# dataList = os.listdir(inputPath)
# dataName = dataList[0][:-7]
# print(dataName)

# skinData.to_csv('Data/' + dataName + 'Skin.csv', index=None, header=None)
# resData.to_csv('Data/' + dataName + 'Res.csv', index=None, header=None)
# ekgData.to_csv('Data/' + dataName + 'EKG.csv', index=None, header=None)

                    TimeStamp  Data
0  2023-11-21-19:38:21.963758  -959
1  2023-11-21-19:38:21.963758  -959
2  2023-11-21-19:38:21.963758  -961
3  2023-11-21-19:38:21.967261  -962
4  2023-11-21-19:38:21.967261  -962
                        TimeStamp  Data
11436  2023-11-21-19:38:21.970523 -3556
11437  2023-11-21-19:38:21.970523 -3556
11438  2023-11-21-19:38:21.970523 -3557
11439  2023-11-21-19:38:21.970523 -3556
11444  2023-11-21-19:38:21.981905 -3556
                    TimeStamp  Data
0  2023-11-21-19:38:21.962261   -14
1  2023-11-21-19:38:21.962261     4
2  2023-11-21-19:38:21.962261    27
3  2023-11-21-19:38:21.962261    25
4  2023-11-21-19:38:21.962261    18
                    TimeStamp  Data
0  2023-11-21-19:38:21.962261   -14
1  2023-11-21-19:38:21.962261     4
2  2023-11-21-19:38:21.962261    27
3  2023-11-21-19:38:21.962261    25
4  2023-11-21-19:38:21.962261    18
                     TimeStamp  Data
28  2023-11-21-19:38:21.962261 -3556
29  2023-11-21-19:38:21.962261 -3556
3